<a href="https://colab.research.google.com/github/kisharnath/AssameseTalk/blob/main/complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!git clone https://github.com/AI4Bharat/IndicTrans2.git

In [ ]:
%%capture
%cd /content/IndicTrans2/huggingface_interface

In [ ]:
%%capture
!python3 -m pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer
!python3 -c "import nltk; nltk.download('punkt')"
!python3 -m pip install bitsandbytes scipy accelerate datasets
!python3 -m pip install sentencepiece

!git clone https://github.com/VarunGumma/IndicTransTokenizer
%cd IndicTransTokenizer
!python3 -m pip install --editable ./
%cd ..

In [ ]:
!git clone https://github.com/VarunGumma/IndicTransTokenizer
%cd IndicTransTokenizer
!python3 -m pip install --editable ./
%cd ..

fatal: destination path 'IndicTransTokenizer' already exists and is not an empty directory.
/content/IndicTrans2/huggingface_interface/IndicTransTokenizer
Obtaining file:///content/IndicTrans2/huggingface_interface/IndicTransTokenizer
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/VarunGumma/indic_nlp_library to /tmp/pip-install-mx0fi_9i/indic-nlp-library-it2_9c2f7ae6c370416b8034adc29ced8529
  Running command git clone --filter=blob:none --quiet https://github.com/VarunGumma/indic_nlp_library /tmp/pip-install-mx0fi_9i/indic-nlp-library-it2_9c2f7ae6c370416b8034adc29ced8529
  Resolved https://github.com/VarunGumma/indic_nlp_library to commit 18e85551b0b331a65d11acf869afd4cfa615bc4c
  Preparing metadata (setup.py) ... done
  Attempting uninstall: IndicTransTokenizer
    Found existing installation: IndicTransTokenizer 0.1.1
    Uninstalling IndicTransTokenizer-0.1.1:
      Successfully uninstalled IndicTransTokenizer-0.1.1
  Running setup.py develop for IndicTransTo

In [ ]:

import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer

BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None

ImportError: cannot import name 'IndicProcessor' from 'IndicTransTokenizer' (unknown location)

In [ ]:
def initialize_model_and_tokenizer(ckpt_dir, direction, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = IndicTransTokenizer(direction=direction)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            src=True,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text
        generated_tokens = tokenizer.batch_decode(generated_tokens.detach().cpu().tolist(), src=False)

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

**English to indic**

In [ ]:
def get_assamese():
  en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
  en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, "en-indic", quantization)

  ip = IndicProcessor(inference=True)

  en_sents = [
      "When I was young, I used to go to the park every day.",
      "He has many old books, which he inherited from his ancestors.",
      "I can't figure out how to solve my problem.",
      "She is very hardworking and intelligent, which is why she got all the good marks.",
      "We watched a new movie last week, which was very inspiring.",
      "If you had met me at that time, we would have gone out to eat.",
      "She went to the market with her sister to buy a new sari.",
      "Raj told me that he is going to his grandmother's house next month.",
      "All the kids were having fun at the party and were eating lots of sweets.",
      "My friend has invited me to his birthday party, and I will give him a gift.",
  ]

  src_lang, tgt_lang = "eng_Latn", "asm_Beng"
  hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

  print(f"\n{src_lang} - {tgt_lang}")
  for input_sentence, translation in zip(en_sents, hi_translations):
      print(f"{src_lang}: {input_sentence}")
      print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
#del en_indic_tokenizer, en_indic_model

**Indic to English**

In [ ]:
def get_english():
  indic_en_ckpt_dir = "ai4bharat/indictrans2-indic-en-1B"  # ai4bharat/indictrans2-indic-en-dist-200M
  indic_en_tokenizer, indic_en_model = initialize_model_and_tokenizer(indic_en_ckpt_dir, "indic-en", "")

  ip = IndicProcessor(inference=True)

  hi_sents = [
      "जब मैं छोटा था, मैं हर रोज़ पार्क जाता था।",
      "उसके पास बहुत सारी पुरानी किताबें हैं, जिन्हें उसने अपने दादा-परदादा से विरासत में पाया।",
      "मुझे समझ में नहीं आ रहा कि मैं अपनी समस्या का समाधान कैसे ढूंढूं।",
      "वह बहुत मेहनती और समझदार है, इसलिए उसे सभी अच्छे मार्क्स मिले।",
      "हमने पिछले सप्ताह एक नई फिल्म देखी जो कि बहुत प्रेरणादायक थी।",
      "अगर तुम मुझे उस समय पास मिलते, तो हम बाहर खाना खाने चलते।",
      "वह अपनी दीदी के साथ बाजार गयी थी ताकि वह नई साड़ी खरीद सके।",
      "राज ने मुझसे कहा कि वह अगले महीने अपनी नानी के घर जा रहा है।",
      "सभी बच्चे पार्टी में मज़ा कर रहे थे और खूब सारी मिठाइयाँ खा रहे थे।",
      "मेरे मित्र ने मुझे उसके जन्मदिन की पार्टी में बुलाया है, और मैं उसे एक तोहफा दूंगा।",
  ]
  src_lang, tgt_lang = "hin_Deva", "eng_Latn"
  en_translations = batch_translate(hi_sents, src_lang, tgt_lang, indic_en_model, indic_en_tokenizer, ip)


  print(f"\n{src_lang} - {tgt_lang}")
  for input_sentence, translation in zip(hi_sents, en_translations):
      print(f"{src_lang}: {input_sentence}")
      print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
#del indic_en_tokenizer, indic_en_model

In [ ]:
get_english()

NameError: name 'IndicTransTokenizer' is not defined

**Now server set up**

In [ ]:
!pip install "uvicorn[standard]" nest-asyncio pyngrok fastapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 9.0 MB/s eta 0:00:00


**Transliteration By Ai4Bharat**

In [ ]:
!pip install ai4bharat-transliteration

In [ ]:
from ai4bharat.transliteration import XlitEngine
e = XlitEngine("as", beam_width=20, rescore=True)
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get('/')
async def root():
    return {'hello': 'world'}
@app.get('/{word}')
async def transalet(word: str):
  out = e.translit_sentence(word)
  print(out)
  return out

MB100% (121.0 of 121.0) |################| Elapsed Time: 0:00:00 Time:  0:00:00


Succefully Downloaded to: /usr/local/lib/python3.10/dist-packages/ai4bharat/transliteration/transformer/models/en2indic/v1.0/model.zip
Models downloaded to: /usr/local/lib/python3.10/dist-packages/ai4bharat/transliteration/transformer/models/en2indic/v1.0
NOTE: When uninstalling this library, REMEMBER to delete the models manually


MB100% (812.0 of 812.0) |################| Elapsed Time: 0:00:11 Time:  0:00:11


Succefully Downloaded to: /usr/local/lib/python3.10/dist-packages/ai4bharat/transliteration/transformer/models/en2indic/v1.0/dicts.zip
Initializing Multilingual model for transliteration


Loading dicts into RAM: 100%|██████████| 1/1 [00:00<00:00,  1.65it/s]


**Run the server**

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn
ngrok.set_auth_token("2dOPQRfVjARgr6oi5CQ7gcvefjr_6pcNtTYXhagk29BmjiLQ")
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [1266]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://b969-104-198-101-95.ngrok-free.app
INFO:     2409:4065:390:fc4:31fc:7728:9f6e:f440:0 - "GET / HTTP/1.1" 200 OK
{'as': 'ফেভিকন.আইচিচি'}
INFO:     2409:4065:390:fc4:31fc:7728:9f6e:f440:0 - "GET /favicon.ico HTTP/1.1" 200 OK
INFO:     2409:4065:390:fc4:31fc:7728:9f6e:f440:0 - "GET / HTTP/1.1" 200 OK
{'as': 'ফেভিকন.আইচিচি'}
INFO:     2409:4065:390:fc4:31fc:7728:9f6e:f440:0 - "GET /favicon.ico HTTP/1.1" 200 OK
{'as': 'তুমি'}
INFO:     2409:4065:390:fc4:31fc:7728:9f6e:f440:0 - "GET /tumi HTTP/1.1" 200 OK
{'as': 'ফেভিকন.আইচিচি'}
INFO:     2409:4065:390:fc4:31fc:7728:9f6e:f440:0 - "GET /favicon.ico HTTP/1.1" 200 OK
INFO:     2409:4065:390:fc4:31fc:7728:9f6e:f440:0 - "OPTIONS /tumi HTTP/1.1" 200 OK
{'as': 'তুমি'}
INFO:     2409:4065:390:fc4:31fc:7728:9f6e:f440:0 - "GET /tumi HTTP/1.1" 200 OK
{'as': 'তুমি'}
INFO:     2409:4065:390:fc4:31fc:7728:9f6e:f440:0 - "GET /tumi HTTP/1.1" 200 OK
{'as': 'তুমি'}
INFO:     2409:4065:390:fc4:31fc:7728:9f6e:f440:0 - "GET /tumi HTTP/1.1" 200

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1266]
